# Sentiment Analysis
In diesem Teil möchten wir uns mit Sentiment Analysis beschäftigen. Vereinfacht gesagt beschäftigt sich Sentiment Analysis damit, natürlichsprachliche Aussagen dahingehend zu bewerten, ob die subjektive Aussage des Sprechers positiv oder negativ wertend gemeint ist.

Zu diesem Zweck haben wir den Datensatz von Sentiment140, einem Projekt der Stanford University, ausgewählt. Er beinhaltet 16 Millionen Tweets, die aufgrund der enthaltenen Emoticons automatisch in positiv und negativ eingeteilt wurden.

Der Datensatz liegt als csv-Datei vor. Zunächst möchten wir uns die darin enthaltene Daten etwas genauer ansehen.

## 1. Aufgabe

1. Laden Sie den Datensatz von http://help.sentiment140.com/for-students
2. Lesen Sie den Datensatz in eine Liste ein, benutzen Sie dazu den csv-reader: https://docs.python.org/3/library/csv.html. Da wir uns nur für die Felder `polarity` und `text` interessieren, sollte die Liste mit den Daten folgendes Format haben : `[(polarity, text),...]`. Positive Tweets haben eine polarity von '4', negative von '0'. Wandeln Sie beim Einlesen diese Werte gleich um in 1 (positiv) und 0 (negativ).

In [ ]:
import csv

with open('../training.1600000.processed.noemoticon.csv', 'r', encoding='iso-8859-1') as f:
    # TODO

3. Um einen Einblick in die Daten zu bekommen und um später ein Modell zur Sentiment Analyse trainieren zu können, sollen die Daten zunächst etwas aufbereitet werden. Da die sinntragenden Elemente in den Tweets die Wörter sind, sollten Sie die Tweets in Wörter aufteilen. Um genau zu sein, ist der Term 'Wörter' hier aus linguistischer Sicht etwas falsch, man spricht eigentlich von Tokens. Daher nennt man das Aufteilen von Text auch Tokenizing und die Funktion, die sowas kann, Tokenizer.
Der allereinfachste Tokenizer ist vermutlich die `split` Methode. Tokenisieren Sie damit die eingelesen Tweets. Am Ende sollten Sie eine Liste `tokenized = [(polarity, [token_1,token_2, ...])]` erhalten.

In [ ]:
tokenized = [] # TODO

4. Abgesehen von natürlichsprachlichen Wörtern sind in Tweets mindestens auch Hashtags, Mentions und Links enthalten. Überlegen Sie sich, ob es Sinn ergibt, alle diese Bestandteile in den Daten in dieser From zu behalten. Begründen Sie kurz Ihre Entscheidungen.
Falls Sie sich entschlossen haben, nicht alle diese Bestandteile zu behalten, filtern Sie dementsprechend Ihre Daten. Die Struktur Ihrer Daten sollte am Ende gleich bleiben: `cleaned = [(polarity, [token_1,...])]`

In [ ]:
cleaned = [] # TODO

5. Zählen Sie die Tokens in Ihrem Datensatz. Benutzen Sie dafür ein Dictionary. Geben Sie die 100 häufigsten Wörter sortiert aus. Was stellen Sie fest? Was müssen Sie zusätzlich noch filtern?

In [ ]:
word_count = {}

## 2. Aufgabe

Wie Eingangs erwähnt, beschäftigt sich Sentiment Analysis damit, eine Äußerung automatisch dahingehend zu klassifizieren,
ob der Inhalt positiv oder negativ gemeint ist.
Im Machine-Learning-Jargon gesprochen hat man es also mit einer binären Klassifikation zu tun. Wir möchten im folgenden Teil ein neuronales Netz trainieren, das entscheiden kann, ob ein Tweet positiv oder negativ gemeint.

Für das Training des neuronalen Netzes möchten wir Keras als Framework benutzen. Keras bietet eine Vereinfachung der Tensorflow-API an, d.h. mit deutlich weniger Aufwand kann man alle Funktionalitäten von Tensorflow benutzen.

Keras bietet zwei unterschiedliche APIs zum Erstellen von neuronalen Netzen an, namentlich _sequential_ und _functional_.
Bei der _sequential_-API wird das Model Schicht für Schicht aufgebaut. Leider kann mit dieser API kein Model aufgebaut werden, das Schichten enthält, die mehr als eine Vorgängerschicht gleichzeitig haben, oder einzelne Schichten wiederbenutzt. Mit der _functional_-API ist dies möglich.

Aktuell liegen unsere Daten zwar in tokenisierter und gesäuberter Form vor, allerdings wird ein neuronales Netz damit sehr wenig anfangen können. Wir müssen unsere Daten also noch etwas weiter vorbereiten.

Als Eingabe soll unser neuronales Netz später Vektoren nehmen, deren einzelne Komponenten alle Wörter darstellen und jeder Eintrag die Anzahl des Wortes in den jeweiligen Tweets. Ein Beispiel:
Zwei Tweets "lorem ipsum" und "foo foo bar", die Vektoren hätten die Länge 4 und für den ersten Tweet wäre der Vektor `[1,1,0,0]`, für den zweiten `[0,0,2,1]`.

1. Befüllen Sie das dictionary `word2idx` so, dass jedes Wort auf einen Index abgebildet wird und die Indizes streng monoton aufsteigend sind. Für das Beispiel oben wäre `word2idx = {"lorem": 0, "ipsum": 1, "foo": 3, "bar": 4}`

In [ ]:
word2idx = {} # TODO

2. Welche Länge werden die Vektoren haben?

In [ ]:
VECTOR_LEN = #TODO

3. Wir könnten mit `numpy` ein Array befüllen, das für jeden der 16 Millionen Tweets einen Vektor wie oben beschrieben enthält. 
Bevor Sie damit beginnen, überschlagen Sie, wieviel Speicherplatz (im Hauptspeicher) ein solches Array belegen würde, wenn jeder Eintrag 32 bit hat. Reicht Ihr Hauptspeicher dafür aus?

In [ ]:
MEMORY = #TODO

4. Um das Problem mit dem zu kleinen Hauptspeicher zu umgehen, bietet Keras die Möglichkeit, anstatt auf einem kompletten Datensatz zu operieren, immer nur kleinere Häppchen abzuarbeiten. Dazu wird ein Python-Generator eingesetzt.
Vervollständigen Sie die Funktion unten, so dass ein Generator entsteht. Die Parameter der Funktion sind:
 * d: tokenisierte und gesäuberte Tweets und Labels
 * w2i: das word2index dictionary
 * batch_size: Anzahl der vektorisierten Tweets, die pro Aufruf zurückgegeben werden sollen.
 
Die benutzen Tweets nacheinander aus `d` gewählt werden und kein Tweet mehrfach zurückgegeben werden.

In [ ]:
import random
import numpy as np

def data_generator(d, w2i, batch_size):
    while True:
        batch_x = np.zeros((batch_size, len(w2i.keys())))
        batch_y = np.zeros((batch_size, 1))
        # TODO
        yield batch_x, batch_y

Sie können Ihren Generator wie folgt ausprobieren:

In [ ]:
gen = data_generator(cleaned, word2idx, 100)

In [ ]:
print(next(gen))

Wir sind nun endlich soweit, unser neuronales Netz aufzubauen. Da unser Netz genau ein hidden Layer hat und auch sonst nicht sonderlich komplex ist, benutzen wir die _Sequential_-API von Keras.

In [ ]:
from keras import Sequential
m = Sequential()

5. Fügen Sie einen _Dense_-Layer dem Netz hinzu, als _hidden units_ können Sie 16 nehmen. Da dies auch der Eingabe-Layer ist, müssen Sie den Parameter `input_shape` definieren. (Siehe auch: https://keras.io/layers/core/)

In [ ]:
m.add(#TODO)

6. Als letzten Layer in unserem neuronalen Netz, fügen Sie einen weiteren _Dense_-Layer hinzu. Dieser Layer dient auch als "Ausgabelayer" Überlegen Sie sich die Anzahl der _hidden units_ (Hinweis: Wie lässt sich unser Machine-Learning-Problem kategorisieren?) Welche _Activation_-Funktion wählen Sie?

In [ ]:
m.add(#TODO)

7. Kompilieren Sie das neuronale Netz. Als `optimizer` können Sie 'adam' benutzen. Wählen Sie eine passende `loss`-Funktion aus. Begründen Sie Ihre Entscheidung. (https://keras.io/models/model/#compile)

In [ ]:
#TODO

8. Bevor Sie nun das neuronale Netz trainieren, teilen Sie noch Ihren Datensatz in zwei Teile auf. Einen Teil zum Trainieren und einen zum Evaluieren. Das Verhältnis der beiden Datensätze sollte 70%:30% sein. Bevor Sie die Daten aufteilen, durchmischen Sie sie mit der `shuffle`-Methode aus dem `random`-Modul. Außerdem sollten Sie die Datenmenge zunächst auf ca. 100000 begrenzen, damit das Training des neuronalen Netzes nicht ewig dauert.

In [ ]:
train = []
evaluation = []
#TODO

9. Wir sind nun soweit das neuronale Netz zu trainieren. Da wir den oben entwickelten Generator einsetzen wollen, verwenden wir dazu die `fit_generator`-Methode. Als `batch_size` können Sie 100 nehmen, für den `epochs`-Parameter 10. Was wählen Sie als `steps_per_epoch`-Parameter? (https://keras.io/models/model/#fit_generator)

In [ ]:
m.fit_generator(#TODO)

10. Während das Netz trainiert wird, können Sie sich Gedanken zur Evaluierung machen:
   * Definieren Sie die üblichen Fehlerklassen (wahr positiv, falsch positiv, wahr negativ, falsch negativ)
   * Eine häufig benutzte Evaluationsmetrik ist die _Accuracy_. Beschreiben Sie dieses Metrik und schreiben Sie die Formel zur Berechnung auf.
   * Warum könnte die _Accuracy_ eine schlechte Metrik sein?
   * Zur Evaluation von binären Klassifikationsproblemen wird in der Literatur gerne _Precision_ und _Recall_ verwendet. Wie sind die beiden Evaluationsmaße definiert? Beschreiben Sie diese Metriken mit eigenen Worten. Schreiben Sie auch die Formeln zur Berechnung auf.
   * Warum könnten _Precision_ und _Recall_ bessere Metriken sein als _Accuracy_?

11. Inzwischen sollte das Netz fertig trainiert sein. Speichern Sie es ab!

In [ ]:
m.save('my_net.h5')

12. Evaluieren Sie ihr Netz mit dem Datensatz, den Sie oben beseite gelegt haben. Benutzen Sie dafür die `predict_classes`-Methode des Models. Berechnen Sie dafür _Precision_, _Recall_ und _Accuracy_. Interpretieren Sie kurz Ihre Ergebnisse. 

In [ ]:
precision = 0.0
recall = 0.0
accuracy = 0.0
#TODO

## Hausaufgabe
1. Trainieren Sie Ihr Netz auf dem großen Datensatz.
2. Verändern Sie die Parameter Ihres Netzes (z.B Anzahl _hidden units_, Anzahl _hidden layers_) und trainieren Sie das Netz erneut (auf dem kleinen Datensatz). Was stellen Sie fest?